# Классификация комментариев по токсичности

**Описание проекта**

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Задача**

Обучить модель классифицировать комментарии на позитивные и негативные. Построить модель со значением метрики качества *F1* не меньше 0.75. 

**План выполнения проекта:**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a>

## Подготовка

### Обзор данных

In [1]:
# импортируем все необходимые библиотеки
import pandas as pd
import numpy as np
import warnings
!pip install transformers
import transformers as ppb
import torch
import re

import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# загрузим датасет и посмотрим на данные
data = pd.read_csv('toxic_comments.csv')

In [3]:
data.head(), data.shape

(   Unnamed: 0                                               text  toxic
 0           0  Explanation\nWhy the edits made under my usern...      0
 1           1  D'aww! He matches this background colour I'm s...      0
 2           2  Hey man, I'm really not trying to edit war. It...      0
 3           3  "\nMore\nI can't make any real suggestions on ...      0
 4           4  You, sir, are my hero. Any chance you remember...      0,
 (159292, 3))

В нашем распоряжении набор данных с разметкой о токсичности правок. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак, лишний столбец unnamed нужно удалить. Комментарии необходимо обработать, очистить от знаков, лемматизировать и токенизировать.

In [4]:
#удалим ненужный признак
data = data.drop('Unnamed: 0', axis=1)

Так как в датасете почти 160000 значений, чтобы провести обработку данных уменьшим датасет и возьмем выборку в 30000 строк.

In [5]:
data = data.sample(n=30000, random_state=1).reset_index(drop=True)

### Лемматизация и очистка текста

**Очистка**

C помощью регулярных выражений из библиотеки RE – это инструмент для поиска слова или числа по шаблону оставим только английский алфавит, удалив разные символы, цифры и т д, которые не несут никакой информации и приведем к нижнему регистру.

In [6]:
# Функция для чистки текста
def clear_text(text):
    t = re.sub(r'[^a-zA-Z ]', ' ', text) 
    t = " ".join(t.split())
    return t

In [7]:
# очистим текст
data['text'] = data['text'].apply(clear_text)

In [8]:
# приведем к нижнему регистру
data['text'] = data['text'].str.lower()

**Лемматизация**

Приведение слова к начальной форме. Для англоязычного текста используем библиотеку **NLTK** и ее функцию для определения **POS-тегов** – частеречной разметки (part-of-speech tagging) — этап автоматической обработки текста, задачей которого является определение части речи и грамматических характеристик слов в тексте с приписыванием им соответствующих тегов.

In [9]:
# функция для определения POS-тегов для слов
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

# проверим работу функции
sentence = "The striped bats are hanging on their feet for best"
print([lemmatizer.lemmatize(w, get_wordnet_pos(w)) 
       for w in nltk.word_tokenize(sentence)])


['The', 'strip', 'bat', 'be', 'hang', 'on', 'their', 'foot', 'for', 'best']


In [10]:
# функция для лемматизации
def lemmatize(text):
    text = list(text.split())
    lemmatizer = WordNetLemmatizer()
    lemm_list = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in text]
    
    return ' '.join(lemm_list)

In [11]:
# лемматизируем
data['text'] = data['text'].apply(lemmatize)

KeyboardInterrupt: 

In [12]:
# проверим результат предобработки
data.head()

,text,toxic
0,bite me that is all cunt,1
1,actually im certain this will be the last brea...,0
2,i protest not for the sake of arguing but for ...,0
3,it also just now occurs to me i didn t mention...,0
4,keep there s an entire category for us weekday...,0


### Разделение на выборки

Разделим выборку на тренировочную и тестовую 80% и 20%.

In [13]:
features = data.drop('toxic', axis=1)
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                            test_size=0.2, random_state=12345)

## Обучение

### Дисбаланс классов

Исследуем баланс классов

In [14]:
target_train.value_counts(), target_test.value_counts()

(0    21536
 1     2464
 Name: toxic, dtype: int64,
 0    5367
 1     633
 Name: toxic, dtype: int64)

Положительных ответов в выборке гораздо меньше, чем отрицательных, исследуем вариант уменьшения и увеличения выборки.

In [15]:
# уменьшим отрицательные ответы, чтобы сбалансировать выборки

def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [16]:
# увеличим положительные ответы, чтобы сбалансировать выборки
def upsample(features, target, repeat):
    features_1 = features[target == 1]
    features_0 = features[target == 0]
    target_1 = target[target == 1]
    target_0 = target[target == 0]
    
    features = pd.concat([features_0] + [features_1] * repeat)
    target = pd.concat([target_0] + [target_1] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features, target, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

### Классификация TFIDF

#### Логистическая регрессия

TF-IDF = TF * IDF

TF = t/n 

t — количество употребления слова, n — общее число слов в тексте

IDF = lg(D/d)

D - общее число текстов в корпусе, d - количество текстов, в которых это слово встречается 

In [17]:
# преобразуем выборки в объект TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', min_df=0.0001, ngram_range=(1, 2)) 

tf_idf_train = vectorizer.fit_transform(features_train['text'])

print ('Размер обучающей выборки: ', tf_idf_train.shape)

Размер обучающей выборки:  (24000, 38524)


In [18]:
# через Пайплайн векторизируем текст и обучим логистическую регрессию
pipeline = Pipeline([("tfidf", TfidfVectorizer()), ('LG', LogisticRegression())])

In [19]:
# добавим параметры моделей
parameters = {'tfidf__stop_words' : (['english']), 'tfidf__min_df' : (0.0001, 0.0002), 'tfidf__norm': ('l1', 'l2'),
             'LG__random_state' : ([12345])}

grid_search = GridSearchCV(pipeline, parameters, cv=3, scoring = 'f1')

In [20]:
# обучим модель
grid_search.fit(features_train['text'], target_train)

# выведем значение F1
print('Значение F1: ', grid_search.best_score_)

Значение F1:  0.5636999569818696


**Метрика F1 показывает довольно низкое качество предсказаний, исследуем качество модели, обученной на сбалансированных выборках**

F1 = 2 * (precision * recall) / (precision + recall)

In [21]:
# обучим модель на уменьшенной выборке
grid_search.fit(features_downsampled['text'], target_downsampled)

# оценим значение метрики F1
print('Значение F1: ', grid_search.best_score_)

Значение F1:  0.7313263103381976


In [22]:
# обучим модель на увеличенной выборке
grid_search.fit(features_upsampled['text'], target_upsampled)

# оценим значение метрики F1
print('Значение F1: ', grid_search.best_score_)

Значение F1:  0.8766504817930239


In [23]:
#  преобразуем сбалансированные выборки в объект TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', min_df=0.0001, ngram_range=(1, 2)) 


tf_idf_train_down = vectorizer.fit_transform(features_downsampled['text'])
tf_idf_train_up = vectorizer.fit_transform(features_upsampled['text'])


print ('Размер уменьшенной выборки: ', tf_idf_train_down.shape)
print ('Размер увеличенной выборки: ', tf_idf_train_up.shape)

Размер уменьшенной выборки:  (7848, 202475)
Размер увеличенной выборки:  (31392, 68197)


Увеличение выборки значительно улучшило качество предсказаний модели. Обучим другие модели.

**Случайный лес**

In [24]:
# обучим случайный лес на небольшой выборке, чтобы подобрать гиперпараметры

best_model = 0
best_score = -1000
    
for depth in range(7, 10):
    model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=20)

    scores = cross_val_score(model, tf_idf_train_up, target_upsampled, cv=4,scoring = 'f1')
    
    print (scores)           
    score = pd.Series(scores).mean()
    
    if score > best_score:
        best_score = score
        best_model = model
            
print ('Лучший результат случайного леса: ', best_score)
print ('Лучшая модель случайного леса', best_model)

[0.0024321  0.00405022 0.01530407 0.01290323]
[0.0024321  0.00566572 0.02167804 0.0288    ]
[0.00969305 0.01370415 0.03273453 0.04827859]
Лучший результат случайного леса:  0.026102581754147473
Лучшая модель случайного леса RandomForestClassifier(max_depth=9, n_estimators=20, random_state=12345)


Случайный лес совсем не подходит для решения данной задачи.

#### BERT

Исследуем модель берт на небольшой выборке в 1000 комментариев.

In [25]:
sample_data = data.sample(n=1000, random_state=1).reset_index(drop=True)

In [26]:
# инициализируем токенизатор
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
bert_model = model_class.from_pretrained(pretrained_weights)

warnings.filterwarnings('ignore')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
# токенизируем текст
tokenized = sample_data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512)))

In [28]:
# найдем максимальную длину векторов
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

# добавим паддинги к векторам
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
padded.shape

(1000, 512)

In [29]:
# создадим маску для важных токенов
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 512)

In [32]:
# преобразуем данные в формат тензоров
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)
input_ids.shape
attention_mask.shape

torch.Size([1000, 512])

In [38]:
%%time

# обучим модель BERT по батчам из двух строк
from tqdm import notebook
batch_size = 2
embeddings = [] 
for i in notebook.tqdm(range(input_ids.shape[0] // batch_size)):
        batch = input_ids[batch_size*i:batch_size*(i+1)]
        attention_mask_batch = attention_mask[batch_size*i:batch_size*(i+1)]
        
        with torch.no_grad():
            batch_embeddings = bert_model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())
        del batch
        del attention_mask_batch
        del batch_embeddings

# сохраним признаки      
features = np.concatenate(embeddings) 

  0%|          | 0/500 [00:00<?, ?it/s]

Wall time: 1h 52min 40s


In [39]:
# сохраним таргет
target = sample_data['toxic']

# разделим на выборки
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.2)

In [40]:
# обучим модель и проведем кросс-валидацию с помощью GridSearch
parameters = {'C': np.linspace(0.0001, 100, 20),'random_state' : ([12345]),
              'max_iter' : ([10000]),
              'solver' : (['saga']), 'penalty' : (['l2'])}

grid_search_bert = GridSearchCV(LogisticRegression(), parameters, scoring='f1', cv=4)
grid_search_bert.fit(train_features, train_target)

# оценим ее качество
print('Лучшие параметры: ', grid_search_bert.best_params_)
print('Лучшее качество: ', grid_search_bert.best_score_)

Лучшие параметры:  {'C': 10.526405263157894, 'max_iter': 10000, 'penalty': 'l2', 'random_state': 12345, 'solver': 'saga'}
Лучшее качество:  0.47380563675630083


Так как выборка для модели BERT у нас небольшая, наилучий результат показала модель Логистической регрессии обученная на увеличенной выборке. Протестируем модели.

### Тестирование

Проверим модель на тестовой выборке.

In [76]:
# предсказания модели BERT
prediction = grid_search_bert.predict(test_features)

print ('F1-мера модели BERT: ', f1_score(test_target, prediction))

F1-мера модели BERT:  0.42424242424242425


In [77]:
# сделаем предсказания логистической регрессии с помощью Пайплайн
pipeline = Pipeline([("tfidf", TfidfVectorizer()), ('LG', LogisticRegression())])

parameters = {'tfidf__stop_words' : (['english']), 'tfidf__min_df' : ([0.0001]),
             'LG__random_state' : ([12345]),
              'LG__C' : ([2]), 'LG__max_iter' : ([10000]),
             'LG__solver' : (['saga']), 'LG__penalty' : (['l2'])}

grid_search = GridSearchCV(pipeline, parameters, cv=3, scoring = 'f1')

In [78]:
# обучим на увеличенной выборке
grid_search.fit(features_upsampled['text'], target_upsampled)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('LG', LogisticRegression())]),
             param_grid={'LG__C': [2], 'LG__max_iter': [10000],
                         'LG__penalty': ['l2'], 'LG__random_state': [12345],
                         'LG__solver': ['saga'], 'tfidf__min_df': [0.0001],
                         'tfidf__stop_words': ['english']},
             scoring='f1')

In [79]:
print (grid_search.best_score_)
print (grid_search.best_params_)

0.9122249280989156
{'LG__C': 2, 'LG__max_iter': 10000, 'LG__penalty': 'l2', 'LG__random_state': 12345, 'LG__solver': 'saga', 'tfidf__min_df': 0.0001, 'tfidf__stop_words': 'english'}


In [80]:
# также проверим качество предсказания модели
prediction = grid_search.predict(features_test['text'])
print ('F1-мера модели TFIDF: ', f1_score(prediction, target_test))


F1-мера модели TFIDF:  0.7508417508417509


Мы достигли цели, довели метрику до 0.75. На тестовой выборке лучший результат показала модель BERT не смотря на то, что она обучалась на небольшой выборке.

## Выводы

Задача проекта - обучить модель со значением метрики качества F1 не меньше 0.75, которая классифицирует комментарии на позитивные и негативные выполнена.

**Нам удалось добиться значения метрики качества F1 на тестовой выборке - 0.42 с помощью BERT и 0.75 с помощью балансировки данных и логистической регрессии**

В исследовании были выполнены следующие шаги:

Подготовлены данные:
* Изучен датасет
* Лемматизация и очистка текста
* Данные разделены на выборки

Обучение модели:
* Обучена логистическая регрессия, выявлен дисбаланс классов
* Проведена балансировка - выявлен лучший метод - Увеличение выборки
* Оценено качество разных методов
* Исследована модель BERT
* Протестированы лучшие модели, получено высокое качество предсказания

Выводы сделаны